<a href="https://colab.research.google.com/github/Ecruzesp/Thesis/blob/main/Random_Forest_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import glob
import cv2 as cv
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
print(tf.__version__)

2.8.2


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
#Capture images and labels into arrays.
#Start by creating empty lists.
images = []
labels = []
for directory_path in glob.glob("/content/drive/MyDrive/Thesis/Transfer_learning/Kaggle/*"):#"Kaggle/three_classes/*"):
    label = directory_path.split("/")[-1]    
    for img_path in glob.glob(os.path.join(directory_path, "*.jpeg")):
        img = cv.imread(img_path, cv.IMREAD_COLOR)
        img = cv.resize(img, (256,256), interpolation=cv.INTER_CUBIC)
        images.append(img)
        labels.append(label)
        
images = np.array(images)
labels = np.array(labels)

In [7]:
#Capture images and labels into arrays.
#Start by creating empty lists.
images_validation = []
labels_validation = []
for directory_path in glob.glob("/content/drive/MyDrive/Thesis/Transfer_learning/Base_Local/*"):
    label = directory_path.split("/")[-1]    
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        img = cv.imread(img_path, cv.IMREAD_COLOR)
        img = cv.resize(img, (256,256), interpolation=cv.INTER_CUBIC)
        images_validation.append(img)
        labels_validation.append(label)
        
images_validation = np.array(images_validation)
labels_validation = np.array(labels_validation)

In [ ]:
images_validation.shape

(45, 128, 128, 3)

In [8]:
# Encode labels from text (folder names) to integers.
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(labels)
le.fit(labels_validation)
labels = le.transform(labels)
labels_validation = le.transform(labels_validation)

In [27]:
X = images
y = labels

In [28]:
# test_size: what proportion of original data is used for test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 1)

In [16]:
#Normalize pixel values to between 0 and 1
X_train, X_test = X_train / 255.0, X_test / 255.0

In [32]:
import pandas as pd
from skimage.filters.rank import entropy
from skimage.filters import sobel
from skimage.morphology import disk
from skimage.restoration import denoise_nl_means, estimate_sigma
import warnings
warnings.filterwarnings("ignore")
# FEATURE EXTRACTOR function
# input shape is (n, x, y, c) - number of images, x, y, and channels
def feature_extractor(dataset):
    
    image_dataset = pd.DataFrame()
    
    for image in range(dataset.shape[0]):  #iterate through each file 
        
      df = pd.DataFrame()  #Temporary data frame to capture information for each loop.
      #Reset dataframe to blank after each loop.
        
      input_img = dataset[image, :,:,:]
      img = np.copy(input_img)
      
      sigma_est = np.mean(estimate_sigma(img, multichannel=True))

      denoise_img = denoise_nl_means(img, h=1.15 * sigma_est, fast_mode=True,
                               patch_size=5, patch_distance=3, multichannel=False)
        
      green = denoise_img[:,:,1]

      from sklearn import preprocessing

      # normalize the data attributes
      norm = preprocessing.normalize(green)
        
      ent = entropy(norm, disk(5))

      sob = sobel(norm)
        
      img_norm_green = norm.reshape(-1)
      df['green'] = img_norm_green       
      img_entropy = ent.reshape(-1)
      df['entropy'] = img_entropy
      img_sobel = sob.reshape(-1)
      df['sobel'] = img_sobel

      image_dataset = image_dataset.append(df)
        
    return image_dataset


In [ ]:
####################################################################
#Extract features from training images
image_features = feature_extractor(X_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: UserWarning: Image dtype is not float. By default denoise_nl_means will assume you want to preserve the range of your image (preserve_range=True). In scikit-image 0.19 this behavior will change to preserve_range=False. To avoid this warning, explicitly specify the preserve_range parameter.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: Possible precision loss converting image of type float64 to uint8 as required by rank filters. Convert manually using skimage.util.img_as_ubyte to silence this warning.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: UserWarning: Image dtype is not float. By default denoise_nl_means will assume you want to preserve the range of your image (preserve_range=True). In scikit-image 0.19 this behavior will change to preserve_range=False. To avoid this warning, explicitl

In [ ]:
#Reshape to a vector for Random Forest / SVM training
image_features = np.expand_dims(image_features, axis=0)
X_for_RF = np.reshape(image_features, (X_train.shape[0], -1))  #Reshape to #images, features

In [ ]:
#Define the classifier
from sklearn.ensemble import RandomForestClassifier
RF_model = RandomForestClassifier(n_estimators = 50, random_state = 42)

In [ ]:
# Fit the model on training data
RF_model.fit(X_for_RF, y_train) #For sklearn no one hot encoding

In [ ]:
#Predict on Test data
#Extract features from test data and reshape, just like training data
test_features = feature_extractor(X_test)
test_features = np.expand_dims(test_features, axis=0)
test_for_RF = np.reshape(test_features, (X_test.shape[0], -1))

In [ ]:
#Predict on test
test_prediction = RF_model.predict(test_for_RF)
#Inverse le transform to get original label back. 
test_prediction = le.inverse_transform(test_prediction)

In [ ]:
#Predict on Validation data
#Extract features from test data and reshape, just like training data
validation_features = feature_extractor(images_validation)
validation_features = np.expand_dims(validation_features, axis=0)
validation_for_RF = np.reshape(validation_features, (images_validation.shape[0], -1))

0.10980393 0.67058825


In [ ]:
#Predict on test
validation_prediction = RF_model.predict(validation_for_RF)
#Inverse le transform to get original label back. 
validation_prediction = le.inverse_transform(validation_prediction)

In [ ]:
#Print overall accuracy
from sklearn import metrics
print ("Accuracy = ", metrics.accuracy_score(y_test, test_prediction))

In [ ]:
#Print confusion matrix
from sklearn.metrics import confusion_matrix
import seaborn as sns
cm = confusion_matrix(y_test, test_prediction)

fig, ax = plt.subplots(figsize=(6,6))         # Sample figsize in inches
sns.set(font_scale=1.6)
sns.heatmap(cm, annot=True, ax=ax)